In [1]:
import pymongo
from pymongo import MongoClient,ReturnDocument
from bson import ObjectId
from transformers import pipeline
import re
import json
import scipy
import numpy as np
from nltk import sent_tokenize
import pickle
from bson.binary import Binary
import scipy
import numpy as np

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')


In [54]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]

query = 'Safe drinking water is essential for healthy human development and survival, but millions of poor people in low-income countries only have access to contaminated drinking water'
query_embedding = model.encode(query)

all_vectors = reports_coll.find(
                                {'collection':'Knowledge Notes','abstract_embeddings': {'$exists':True}},
                                {"abstract_embeddings": 1},
                                )
ids_of_interest = []
all_vectors_list = []
for element in all_vectors:
    vector_set = pickle.loads(element["abstract_embeddings"])
    cossimil_abstract_query = [scipy.spatial.distance.cosine(query_embedding,vector) for vector in vector_set]
    if np.mean(cossimil_abstract_query) > 0.8 and max(cossimil_abstract_query)>.8:
        ids_of_interest.append(element['_id'])

print(len(ids_of_interest))
#idx_of_interest = []
#for idx,vectorset in enumerate(all_vectors_list):
#    if np.mean(vectorset) > 0.1 and max(vectorset)>.1:
#        idx_of_interest.append(idx)
#
#all_vectors = reports_coll.find({'abstract_embeddings': {'$exists':True}})
#titles = [all_vectors[idx]["title"] for idx in idx_of_interest if "country_mentions" in all_vectors[idx].keys()]


20


In [55]:
test = reports_coll.find({"_id":{'$in':ids_of_interest}})

dict_mentions = {}
for elem in test:
    if 'country_mentions' in elem.keys():
        for key,value in elem['country_mentions'].items():
            if key in dict_mentions.keys():
                dict_mentions[key]+=value
            elif key not in dict_mentions.keys():
                dict_mentions[key] = value

most_mentioned = sorted(dict_mentions, key=dict_mentions.get,reverse=True)
print(most_mentioned)
#print(sorted(dict_mentions, key=dict_mentions.get,reverse=True))
{k: v for k, v in sorted(dict_mentions.items(), key=lambda item: item[1],reverse=True)}

['china', 'india', 'brazil', 'philippines', 'peru', 'korea', 'samoa', 'tonga', 'georgia', 'mexico', 'kiribati', 'indonesia', 'thailand', 'malaysia', 'myanmar', 'dominica', 'mongolia', 'colombia', 'argentina', 'libya', 'djibouti', 'chile', 'yemen', 'kenya', 'fiji', 'haiti', 'venezuela', 'turkey', 'guinea', 'belarus', 'vanuatu', 'ecuador', 'paraguay', 'honduras', 'dominican republic', 'lithuania', 'solomon islands', 'guatemala', 'azerbaijan', 'serbia', 'cambodia', 'costa rica', 'bolivia', 'cuba', 'antigua and barbuda', 'grenada', 'jamaica', 'lebanon', 'france', 'burkina faso', 'tuvalu', 'panama', 'kazakhstan', 'papua new guinea', 'nicaragua', 'uzbekistan', 'albania', 'armenia', 'turkmenistan', 'ukraine', 'latvia', 'belize', 'uruguay', 'tajikistan', 'bulgaria', 'australia', 'mozambique', 'oman', 'micronesia', 'suriname', 'montenegro', 'syria', 'sudan', 'romania', 'nepal', 'south africa', 'russian federation', 'timor-leste', 'guyana', 'germany', 'japan', 'egypt', 'malawi', 'morocco', 'mars

{'china': 38,
 'india': 27,
 'brazil': 25,
 'philippines': 22,
 'peru': 22,
 'korea': 20,
 'samoa': 20,
 'tonga': 19,
 'georgia': 18,
 'mexico': 18,
 'kiribati': 18,
 'indonesia': 18,
 'thailand': 17,
 'malaysia': 16,
 'myanmar': 14,
 'dominica': 14,
 'mongolia': 14,
 'colombia': 14,
 'argentina': 14,
 'libya': 13,
 'djibouti': 13,
 'chile': 13,
 'yemen': 12,
 'kenya': 12,
 'fiji': 12,
 'haiti': 12,
 'venezuela': 11,
 'turkey': 11,
 'guinea': 10,
 'belarus': 10,
 'vanuatu': 10,
 'ecuador': 10,
 'paraguay': 10,
 'honduras': 10,
 'dominican republic': 9,
 'lithuania': 9,
 'solomon islands': 9,
 'guatemala': 9,
 'azerbaijan': 9,
 'serbia': 9,
 'cambodia': 9,
 'costa rica': 9,
 'bolivia': 9,
 'cuba': 8,
 'antigua and barbuda': 8,
 'grenada': 8,
 'jamaica': 8,
 'lebanon': 7,
 'france': 7,
 'burkina faso': 7,
 'tuvalu': 7,
 'panama': 7,
 'kazakhstan': 7,
 'papua new guinea': 7,
 'nicaragua': 7,
 'uzbekistan': 7,
 'albania': 7,
 'armenia': 7,
 'turkmenistan': 7,
 'ukraine': 7,
 'latvia': 6,
 

In [56]:
test = reports_coll.find({"_id":{'$in' :ids_of_interest}})
for elem in test:
    if 'country_mentions' in elem.keys():
        print(elem['abstract'])

A wide range of new technologies are progressively being used for data collection, project planning, implementation, and monitoring and evaluation in forced displacement contexts. This note highlights case studies from countries in the Middle East and North Africa Region and provides practitioners with specific examples of technology use.
This document provides introduction by providing overall guidance on the use of terms and abbreviations across the entire notes series. Common overarching principles on responsible agricultural investment which underlie the approach and tenor of the series are also introduced. Note 2 complements others in the series by offering additional resources over and above those provided in each of them.
Connections is a series of concise knowledge notes from the World Bank Group’s Transport and ICT Global Practice. Connections discusses projects, experiences, and front-line developments in Transport and ICT. This set includes notes from 2015 and 2016.
Internat

In [6]:
all_vectors.count()

<ipython-input-6-84439e7e48f2>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  all_vectors.count()


8263

In [ ]:
for idx in idx_of_interest:
    print(all_vectors[idx].keys())

In [ ]:
all_vectors[12].keys()

In [ ]:
reports_coll.count_documents({'collection':'Knowledge Notes',
                                          'country_mentions': {'$exists':True},
                                         'abstract_embeddings': {'$exists':True}})                                         



In [ ]:
all_vectors[17]

In [ ]:
idx_of_interest = []
for idx,vectorset in enumerate(all_vectors_list):
    if np.mean(vectorset) > 0.8 and max(vectorset)>.9:
        idx_of_interest.append(idx)
idx_of_interest

In [ ]:
all_vectors = reports_coll.find({'abstract_embeddings': {'$exists':True}})
info = [all_vectors[idx]["title"] for idx in idx_of_interest]
info

In [ ]:
len_stuff = [len(vector) for vector in all_vectors_list]
np.mean(len_stuff)

In [ ]:
lst_vector_scores = []
for vector_set in all_vectors:
    cossimil_abstract_query = [scipy.spatial.distance.cosine(query_embedding,vector) for vector in pickle.loads(vector_set)]
    
